In [2]:
from numpy import exp, array, random, dot
import numpy as np
from scipy import misc
import skimage
import math
from __future__ import with_statement

In [3]:
arr1 = np.divide(misc.imread('save_save_america01.jpg'),255)
arr1r = arr1[:,:,0]
arr1g = arr1[:,:,1]
arr1b = arr1[:,:,2]
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [4]:
filterNum = 1
filterSize = 10
layerNum = 6
epochs = 10
filters = []

In [5]:
def setFilters():
    global filters
    filters = 0.01*random.random((filterNum*layerNum, filterSize, filterSize))-0.005
setFilters()
print(filters)

[[[ -3.26790755e-03   1.43248166e-03  -2.29412385e-03   2.13492156e-03
     7.42880558e-04  -2.68283045e-04   3.78113180e-04  -1.80236499e-03
     2.98290305e-03  -3.66840871e-03]
  [ -3.61127348e-03  -3.66415592e-04  -3.15238252e-04  -7.50172669e-04
     4.39007888e-03  -1.91462183e-03   4.10758793e-03   1.07761349e-03
    -3.59130045e-03  -4.02956628e-03]
  [  2.97290938e-03  -3.26873155e-03   2.26357964e-04   3.41742366e-03
     2.64012845e-03  -2.46752279e-03   1.85163963e-03  -3.71922879e-03
    -2.48451582e-03   1.00343748e-03]
  [  2.29050521e-03   6.19652687e-04  -3.06237758e-03  -3.11877867e-03
    -2.81290871e-03  -2.66373364e-03   3.21987340e-05  -3.22985061e-03
     3.26482928e-03   2.91721001e-03]
  [  4.80079588e-03   2.17356720e-03  -5.29404505e-04  -3.86030262e-03
    -6.32630716e-04  -3.39451140e-03  -4.54506390e-03   1.56080692e-03
     4.05948798e-03   1.52779936e-03]
  [ -2.15040014e-03  -3.70204219e-03   1.04739910e-03   3.51639648e-03
     1.82012225e-03  -2.22180

In [23]:
def activation(x, deriv=False, leak=-0.01, arr=False):
    new_x=np.copy(array(x))
    if deriv:
        new_x[x > 0] = 1
        new_x[x < 0] = leak
    if arr:
        new_x[x < 0] *= leak
        new_x = x * (x > 0)
        return new_x
    new_x = x * (x > 0)
    if new_x==0:
        new_x=leak
    return new_x
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
def apply(filterArr, img, row, col):
    value = 0
    halfFil = int(filterSize/2)
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0))
    for y in range(len(filterArr)):
        for x in range(len(filterArr[0])):
            value += padImg[y+row-halfFil][x+col-halfFil]*filterArr[y][x]
    if sumNoNeg(filterArr)==0:
        return 0
    return value
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)
print(apply(filters[4],arr1r,0,0))

0.0


In [24]:
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])):
            new_img[y][x] = activation(apply(filterArr,img,y,x))
    return new_img

In [25]:
def forward(img):
    outputLayers = [[img]]
    for num in range(layerNum):
        outputLayers.append([])
        for fil in range(filterNum):
            newImage = np.zeros((len(outputLayers[num][fil]), len(outputLayers[num][fil][0])))
            applyFilters(outputLayers[num][fil], newImage, filters[filterNum*num + fil])
            print(filterNum*num + fil)
            print(newImage)
            outputLayers[num+1].append(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten()))
    return outputLayers
out = forward(arr1r)
print(out)

0
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02   2.32643110e-05  -1.00000000e-02 ...,   1.73659102e-04
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02   1.61239431e-05   2.84762047e-05 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 ..., 
 [ -1.00000000e-02   8.40169171e-03   6.34022163e-03 ...,  -1.00000000e-02
   -1.00000000e-02   2.05658422e-03]
 [ -1.00000000e-02   5.63343149e-03   6.16606523e-03 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02   3.52944063e-03   7.26949878e-03 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]]
1
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02   2.90313859e-05   5.88528852e-05 ...,   5.81683144e-05
    5.81683144e-05   5.81683144e-05]
 [ -1.00000000e-02   4.55470488e-05   6.14193677e-06 ...,   8.53444232e-05
   -1.00000000e-0

In [ ]:
def maxPool(size, img):
    new_img = np.zeros((int(len(img)/size), int(len(img[0])/size)))
    pad_img = np.pad(img, [(0, size-len(img)%size),(0, size-len(img[0])%size)], "constant", constant_values=(0))
    for x in range(math.ceil(len(img)/size)-1):
        for y in range(math.ceil(len(img[0])/size)-1):
            new_img[x][y] = np.amax(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
            print(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
    return new_img
print(maxPool(3, out[2]))

In [26]:
error = 1-np.mean(out[len(out)-1])
# def back(img, startlayer, error):
#     print(startlayer)
#     newout = np.zeros((len(out[startlayer-1]), len(out[startlayer-1][0])))
#     if startlayer == -1:
#         return out[0]
#     delta = applyFilters(back(img, startlayer-1, error),
#                  newout,
#                  np.multiply(np.rot90(np.rot90(filters[startlayer])),activation(img,deriv=True)[:filterSize,:filterSize]))
#     filters[startlayer]=np.add(filters[startlayer],array(delta[:filterSize,:filterSize]))
#     print(startlayer)
#     return newout
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
        for row in filters[filterNum*layer+x-1]:
            c=0
            for w in row:
                new_grad[x][r][c]=-np.multiply(error*10,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer-1]),deriv=True,arr=True),
                    w),
                    grad[x][r][c])))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[filterNum*layer+x-1][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

6
5
4
3
2
1


In [28]:
for x in range(epochs):
    out = forward(arr1r)
    error = 1-np.mean(out[len(out)-1])
    print(error)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))

0
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02   6.96378465e-05]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02   1.56251783e-04]
 ..., 
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02   7.11332929e-04 ...,   8.13456859e-03
    1.19016409e-02   8.83677533e-03]
 [ -1.00000000e-02   1.27609283e-03   2.85923745e-03 ...,   1.45389254e-02
    1.74496187e-02   1.33995265e-02]]
1
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02   4.98374952e-06 ...,   7.96337160e-05
    7.96337160e-05   7.96337160e-05]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,   2.22822067e-05
    2.22822067e-0

In [27]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'warn', 'under': 'ignore'}